In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pyfftw
from pyfftw.interfaces.numpy_fft import fftshift
from pyfftw.interfaces.numpy_fft import ifftshift
from pyfftw.interfaces.numpy_fft import rfftn
from pyfftw.interfaces.numpy_fft import irfftn
from pyfftw.interfaces.numpy_fft import rfft2
from pyfftw.interfaces.numpy_fft import irfft2
import scipy as sp
import seaborn as sns

from pyem.ctf import ctf_freqs
from pyem.ctf import eval_ctf
from pyem import mrc
from pyem.util import *
from pyem.vop import *

%matplotlib inline
sns.set()
pyfftw.interfaces.cache.enable()

In [1]:
def radial_sum(data, center=(0,0), r=None):
    if r is None:
        y, x = np.indices((data.shape))
        r = np.sqrt((x - center[0])**2 + (y - center[1])**2)
        r = r.astype(np.int)
#     radialsum = sp.stats.binned_statistic(r.ravel(), data.ravel(), "sum", 192)
    tbin = np.bincount(r.ravel(), data.ravel())
    return tbin

In [ ]:
def fourier_ring_correlation(p1, p2, r=None):
    center = (0, p1.shape[0] // 2)
    fc = p1 * np.conj(p2)
    fcr = radial_sum(np.real(fc), center, r)
    fcc = radial_sum(np.imag(fc), center, r)
    mag = np.sqrt(radial_sum(np.abs(p1)**2, center, r) * 
                  radial_sum(np.abs(p2)**2, center, r))
    frc = (fcr + fcc*1j) / mag
    return frc

In [ ]:
center = (0,96)
y, x = np.indices((p1.shape))
r = np.sqrt((x - center[0])**2 + (y - center[1])**2)
r = r.astype(np.int)
# rbin = (20* r/r.max()).astype(np.int)
r = fftshift(r, axes=0)  # Pre-shift r, leave FFTs shifted.
rflat = r.reshape(-1)

In [ ]:
p1flat = p1.reshape(-1)
p2flat = p2.reshape(-1)
fc = p1flat * np.conj(p2flat)
fcr = np.bincount(rflat, np.real(fc))
fcc = np.bincount(rflat, np.imag(fc))
mag = np.sqrt(np.bincount(rflat, np.abs(p1flat)**2) * 
              np.bincount(rflat, np.abs(p2flat)**2))
frc = (fcr + fcc*1j) / mag

In [ ]:
c = ctf(args)

In [ ]:
# a.flat -= b.ravel() * c.ravel() * frc[r.ravel()]
a -= b * c * frc[r]

In [ ]:
p1 = rfft2(exp_image)
pshift = np.exp(-2 * np.pi * 1j * (xshift * sx + yshift * sy))
p2 = interpolate_slice_numba(submap) * pshift
p3 = interpolate_slice_numba(refmap) * pshift
c = ctf(exp_image_params)
frc = fourier_ring_correlation(p1, p3 * c)
p1 -= p2 * c * frc[r]
new_image = irfft2(p1)